In [1]:
import tensorflow as tf
import numpy as np
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [2]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='<s>', eos_token='</s>', pad_token='<pad>')
gpt_model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.7.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'lm_head.weight', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expe

In [3]:
input_ids = np.array([tokenizer.encode('이때')])
print(type(input_ids))
print(input_ids.shape)
print(input_ids)

<class 'numpy.ndarray'>
(1, 1)
[[9964]]


In [4]:
input_ids = np.array([tokenizer.encode('이때')])
# print(type(input_ids))
output = gpt_model.generate(input_ids, max_length=100, do_sample=True, top_k=20)
# print(output[0].numpy().tolist())
sentence = tokenizer.decode(output[0].numpy().tolist())
print(sentence)

이때 이게 다 뭐랄까, 네. 이거 지금 그~ 김정은이 인제 뭐~ 네. 이~ 뭔가 좀 어~ 어~ 뭔가 뭐~ 어~ 뭔가 이~ 어~ 뭔가 좀 네. 어~ 이게 이~ 뭐~ 이렇게 좀 이렇게 뭐~ <unk>니까?
예. 뭔가 이제 이렇게 뭐~ 이~ 어~ 좀 이~ 뭔가 이런 어~ 뭔가 이~ 뭐~ 


In [5]:
input_ids = np.array([tokenizer.encode('이때')])
# print(type(input_ids))
output = gpt_model.generate(input_ids, max_length=100, do_sample=True, top_p=0.95)
sentence = tokenizer.decode(output[0].numpy().tolist())
print(sentence)

이때도 한국이 중국(러시아)의 편에 서진 않을 것이라는 점도 강조했다.
이런 가운데 일본 NHK는 이날 아베 총리가 중국과 대만을 잇달아 방문하고 있고, 중국이 일본을 향해 경제보복 공세를 강화했다는 보도에 대해 "중국 외교부 장관과 왕이 중국 외교부장을 만나면 이런 말을 들을 수도 있을 것"이라고 예상했다.
일본과 미국을 중심으로 한 안보 동맹에서 중국이 부상하는 상황에서, 이번 사안에 대한 중국의 반응은 "한국과 대만에 대한 경제 보복"이라고 강하게 비난한 것이다.
이날


In [6]:
DATA_IN_PATH = './data_in/KOR/'
TRAIN_DATA_FILE = 'finetune_data.txt'

sents = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE, encoding='utf-8').readlines()]
print(sents[0:2])
print(len(sents))

['그때에 김첨지는 대수롭지 않은듯이,', '만일 김첨지가 주기를 띠지 않았던들 한 발을 대문에 들여놓았을 제 그곳을 지배하는 무시무시한 정적(靜寂) ― 폭풍우가 지나간 뒤의 바다 같은 정적이 다리가 떨렸으리라.']
284


In [7]:
def get_data():
  for sent in sents:
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode(sent ) 
    yield bos_token + sent + eos_token

In [8]:
dataset = tf.data.Dataset.from_generator(get_data, output_types=tf.int32)
dataset = dataset.padded_batch(batch_size=8, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

In [9]:
for batch in dataset:
    print(batch.shape)
    print(batch)
    break

(8, 52)
tf.Tensor(
[[    0 12858  8022  9324  8364  9272 20897 33336 10091 23272     1     3
      3     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3]
 [    0 19089  9324  8364  9737 42414 10481  8263 18318  7285  9036 19846
   9026 10924 15994  7128 23928  9037 35433 21299 15218  7556 27232  9040
  11360  6433  2010   384   739   679 39798 15576 50866 16358 11099  9239
   9040 10018 23948  9940  7422 17082 25462     1     3     3     3     3
      3     3     3     3]
 [    0 30181  9337  9164 13530 41732  9080  9548  9784  8139  7513  9989
   9079 28936   739  6976 22506 27533  7763  8463  7235 24257  8420 10171
   7182 45006  9208  6889  8689  9796  9056  7791  9129  7395  9366  9818
   7198  7321  9174 20579 19897 11520  9676  9220  8006  8528  8137  9036
  15059  8367 18005     1]
 [    0  967

In [10]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [11]:
steps = len(sents) // 8 + 1
print(steps)

36


In [12]:
import tqdm
EPOCHS = 10

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.notebook.tqdm(dataset, total=steps):
      with tf.GradientTape() as tape:
          result = gpt_model(batch, labels=batch)
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)
          
      grads = tape.gradient(batch_loss, gpt_model.trainable_variables)
      adam.apply_gradients(zip(grads, gpt_model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

  0%|          | 0/36 [00:00<?, ?it/s]

[Epoch:    1] cost = 2.90381145


  0%|          | 0/36 [00:00<?, ?it/s]

[Epoch:    2] cost = 1.60398388


  0%|          | 0/36 [00:00<?, ?it/s]

...

[Epoch:    8] cost = 0.250238895


  0%|          | 0/36 [00:00<?, ?it/s]

[Epoch:    9] cost = 0.231242761


  0%|          | 0/36 [00:00<?, ?it/s]

[Epoch:   10] cost = 0.221696824


In [13]:
import os
DATA_OUT_PATH = './data_out'
model_name = "tf2_gpt2_finetuned_model"

save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

gpt_model.save_weights(save_path)

In [15]:
input_ids = np.array([tokenizer.encode('이때')])
output = gpt_model.generate(input_ids, max_length=100)
sentence = tokenizer.decode(output[0].numpy().tolist(),skip_special_tokens=True)
print(sentence)

이때 “에미를 붙을 이 오라질 놈들 같으니, 이놈 내가 돈이 없을 줄 알고.”


In [19]:
input_ids = np.array([tokenizer.encode('김첨지')])
output = gpt_model.generate(input_ids, max_length=100, do_sample=True, top_p=0.95)
sentence = tokenizer.decode(output[0].numpy().tolist(),skip_special_tokens=True)
print(sentence)

김첨지사는 구걸하는 거지나 무엇같이 연해연방 그의 기색을 살피며,
